In [169]:
!pip install -q -U google-genai
!pip install pydantic

In [170]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("eriktks/conll2003")

# Access the train, validation, and test splits
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

# Print a sample
print(train_data[0])

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [171]:
# Print a sample from the training data
example = train_data[11001]
print("Example from the training data:")
print(example)
print("Tokens:", example["tokens"])
print("Labels:", example["ner_tags"])

Example from the training data:
{'id': '11001', 'tokens': ['1886', '-', 'At', 'Skeleton', 'Canyon', 'in', 'Arizona', ',', 'Geronimo', ',', 'Apache', 'chief', 'and', 'leader', 'of', 'the', 'last', 'great', 'Red', 'Indian', 'rebellion', 'finally', 'surrendered', 'to', 'General', 'Nelson', 'Miles', '.'], 'pos_tags': [11, 8, 15, 22, 22, 15, 22, 6, 22, 6, 22, 21, 10, 21, 15, 12, 16, 16, 22, 22, 21, 30, 38, 35, 22, 22, 22, 7], 'chunk_tags': [11, 12, 13, 11, 12, 13, 11, 0, 11, 0, 11, 12, 0, 11, 13, 11, 12, 12, 12, 12, 12, 3, 21, 13, 11, 12, 12, 0], 'ner_tags': [0, 0, 0, 5, 6, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 1, 2, 0]}
Tokens: ['1886', '-', 'At', 'Skeleton', 'Canyon', 'in', 'Arizona', ',', 'Geronimo', ',', 'Apache', 'chief', 'and', 'leader', 'of', 'the', 'last', 'great', 'Red', 'Indian', 'rebellion', 'finally', 'surrendered', 'to', 'General', 'Nelson', 'Miles', '.']
Labels: [0, 0, 0, 5, 6, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 1, 2, 0]


In [172]:
print("Training set size:", len(train_data))
print("Validation set size:", len(validation_data))
print("Test set size:", len(test_data))

Training set size: 14041
Validation set size: 3250
Test set size: 3453


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [173]:
import csv
import os

'''
Store the response in a list of lists where the first element is the token, the second element 
is the predicted label and the third is the true label
'''
def parse_response(tokens : list, response_labels : list, true_labels : list) -> list: 
    response_labels = response_labels.split(":")
    response_labels = response_labels[1].strip('\n').split(',')
    assert (len(response_labels) == len(tokens)), "Length of tokens and NER tags do not match"

    temp = []
    for i in range(len(tokens)):
        pred_label = int(response_labels[i].strip())
        
        assert (pred_label >= 0 and pred_label <= 8), "Predicted label is out of range"
        temp.append([tokens[i], pred_label, true_labels[i]])

    return temp

def save_to_csv(tokens : list, pred_labels : list, true_labels : list, filename : str) -> None:
    file_exists = os.path.isfile(filename)
    # Write header only if the file didn't exist before
    if not file_exists:
        with open(filename, 'a', newline='') as csvfile:
            header = ['token', 'pred', 'true']
            writer = csv.writer(csvfile)
            writer.writerow(header)
    data = [[tokens[i], pred_labels[i], true_labels[i]] for i in range(len(tokens))]
    # Open the file in append mode and write data to analysis purpose
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

In [174]:
# Take predicted labels and for each token save the label in a list to be used for voting
def store_predicted_labels(pred_labels : list, votes : list) -> None:
    for i in range(len(pred_labels)):
        votes[i].append(pred_labels[i])

In [175]:
#from pydantic import BaseModel
from google import genai

for j in range(5):
    tokens = train_data[j]['tokens']
    true_labels = train_data[j]['ner_tags']
    votes = [[] for _ in range(len(tokens))]
    model = "gemma-3-27b-it"
    for i in range(5):
        client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
        # Send request to Gemma
        response = client.models.generate_content(
            model=model,
            contents=f"""Given the following NER tags: {{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}}, determine the Named Entity Recognition (NER) tags for the following sentence.
            The sentence is: '{tokens}'
            This sentence contains exactly {len(tokens)} tokens.

            Print only the number associated with the NER tag for each of the {len(tokens)} tokens, using the tag-to-number mapping provided above.
            Your answer MUST follow the format: ner_tags: 0, 1, 2, 0, 0, 0
            The number of output NER tags MUST be exactly {len(tokens)}, one for each token in the order they appear in the sentence.
            Do not include any other text or explanations.
            """
        )

        # Parse the response
        data = parse_response(tokens, response.text, true_labels)
        # Store predicted labels for voting
        store_predicted_labels([data[i][1] for i in range(len(data))], votes)
    # Extract for each token the most voted label
    votes = [max(set(vote), key=vote.count) for vote in votes]
    save_to_csv(tokens, votes, true_labels, "data/data_test.csv")